In [11]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import fbeta_score

data = np.load('positional_all_players_tensor.npy')
data2 = np.load('noninjured_positional_all_players_tensor.npy')
data = np.concatenate((data, data2),axis=0)



X = data[:, :, :]
X = np.delete(X, -2, axis=2)
y = data[:, :, -2] # Labels: second last column indicating injury (1) or not (0)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

y_train_aggregated = y_train[:, -1].unsqueeze(1)  




class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out


input_size = X.shape[2]  # Number of features
hidden_size = 128
num_layers = 4
output_size = 1


model = RNN(input_size, hidden_size, num_layers, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training the model
num_epochs = 10


for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        labels = labels[:, -1]  # Selecting only the last label in each sequence
        loss = criterion(outputs.squeeze(), labels)
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')



from sklearn.metrics import f1_score


with torch.no_grad():
    all_player_predictions = []
    true_labels_flat = []
    
    for i in range(len(X_test)): 
        player_predictions = []
        true_labels_match = []  # Collect true labels for this match
        for j in range(X_test.size(1)):  

            input_match = X_test[i:i+1, j:j+1, :]
            
            output = model(input_match)
            predicted = torch.round(torch.sigmoid(output)).item()
            
            player_predictions.append(predicted)
            
            actual = int(y_test[i, j+1].item()) if j < X_test.size(1) - 1 else None
            if actual is not None:
                true_labels_match.append(actual)
        
        all_player_predictions.append(player_predictions)
        true_labels_flat.extend(true_labels_match)  

    pred_labels_flat = [label for sublist in all_player_predictions for label in sublist]


    if len(pred_labels_flat) > len(true_labels_flat):
        pred_labels_flat = pred_labels_flat[:len(true_labels_flat)]
    elif len(pred_labels_flat) < len(true_labels_flat):
        true_labels_flat = true_labels_flat[:len(pred_labels_flat)]

    true_labels_flat = np.array(true_labels_flat)
    pred_labels_flat = np.array(pred_labels_flat)

    f2 = fbeta_score(true_labels_flat, pred_labels_flat, beta=2, average='weighted')

    print(f'F1 Score on test set: {f1:.2f}')
 



# Check sigmoid functions of each class, quantify the risk of injury instead of just uninjured or injury
# Try changing loss function



Epoch [1/10], Step [10/30], Loss: 0.0082
Epoch [1/10], Step [20/30], Loss: 0.0017
Epoch [1/10], Step [30/30], Loss: 0.0009
Epoch [2/10], Step [10/30], Loss: 0.0006
Epoch [2/10], Step [20/30], Loss: -0.5005
Epoch [2/10], Step [30/30], Loss: 0.0002
Epoch [3/10], Step [10/30], Loss: 0.0001
Epoch [3/10], Step [20/30], Loss: 0.0001
Epoch [3/10], Step [30/30], Loss: 0.0001
Epoch [4/10], Step [10/30], Loss: 0.0000
Epoch [4/10], Step [20/30], Loss: 0.0000
Epoch [4/10], Step [30/30], Loss: 0.0000
Epoch [5/10], Step [10/30], Loss: 0.0000
Epoch [5/10], Step [20/30], Loss: 0.0000
Epoch [5/10], Step [30/30], Loss: 0.0000
Epoch [6/10], Step [10/30], Loss: -0.8143
Epoch [6/10], Step [20/30], Loss: 0.0000
Epoch [6/10], Step [30/30], Loss: 0.0000
Epoch [7/10], Step [10/30], Loss: 0.0000
Epoch [7/10], Step [20/30], Loss: 0.0000
Epoch [7/10], Step [30/30], Loss: 0.0000
Epoch [8/10], Step [10/30], Loss: 0.0000
Epoch [8/10], Step [20/30], Loss: 0.0000
Epoch [8/10], Step [30/30], Loss: 0.0000
Epoch [9/10], 